# Proyecto 1 - Analítica de textos 
## Análisis de sentimientos en películas

### Integrantes:
- Daniel Aguilera
- Vanessa Martínez
- Cristian Sánchez

### Descripción del problema
El objetivo de este notebook es realizar el desarrollo de la etapa 2 para el cual, se llevara acabo el analisis de sentimiento de las opiniones de las peliculas que se encuentran en el dataset MovieReviews.csv, para este fin se utilizara el modelo de Naive Bayes para clasificar las opiniones en positivas y negativas. 

Se usara el modelo Naive Bayes ya que puede ofrecer resultados muy precisos en el análisis de sentimientos. Lo anterior se debe a que es capaz de identificar patrones en los datos y aprender de ellos para mejorar su rendimiento. Además, el modelo Naive Bayes funciona bien incluso cuando se tienen datos desbalanceados, lo que es común en el análisis de sentimientos.

## Referencias
1. [Datos a utilizar: MovieReviews.csv](./datos/MovieReviews.csv)
2. [Diccionario de datos: DiccionarioPeliculas.xlsx](./datos/DiccionarioPeliculas.xlsx)
3. [Datos de prueba: MovieReviewsPruebas.csv](./datos/MovieReviewsPruebas.csv)
4. [Datos procesados: MovieReviewsProcesado.csv](./datos/MovieReviewsProcesado.csv)
5. [Notebook de procesamiento y entendimiento de datos](./proy_desarrollo.ipynb)
6. [Información del negocio](./datos/enunciado.pdf)

In [197]:
# Realizamos los imports necesarios
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Importamos MLP para crear la red neuronal
from sklearn.neural_network import MLPClassifier

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report, precision_score, recall_score, f1_score, accuracy_score
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.feature_extraction.text import CountVectorizer

# Import nltk
import nltk

# Usamos las stopwords de nltk
from nltk.corpus import stopwords
stop_words = stopwords.words('spanish')

### Una vez importadas las librerias necesarias, se procede a cargar los datos preprocessados en el notebook anterior

In [198]:
# Cargamos el dataset
df_peliculas = pd.read_csv('./datos/MovieReviewsProcesado.csv')

# Mostramos las primeras filas
df_peliculas.head()

,Unnamed: 0,review_es,sentimiento,processed_text
0,0,Si está buscando una película de guerra típica...,1,si busc pelicul guerr tipic asi not aficion gu...
1,1,Supongo que algunos directores de películas de...,1,supon director pelicul luj sent busc abrig gra...
2,2,Es difícil contarle más sobre esta película si...,1,dificil contar el mas pelicul estropearlal dis...
3,3,"La película comienza muy lentamente, con el es...",1,pelicul comenz lent estil vid wallac napalm as...
4,4,Esta película es verdadera acción en su máxima...,1,pelicul verdader accion maxim expresion mejor ...


## Modelo Naive Bayes

EL Modelo Naive Bayes es una buena opción para el objetivo del proyecto debido a su capacidad para trabajar con datos de texto. La clasificación de comentarios de películas se puede ver como una tarea de clasificación de texto, en la que los comentarios son el texto y la etiqueta de "positivo" o "negativo" es la clase a predecir.

### Comenzamos a construir el modelo

El conjunto de datos se divide en partes de entrenamiento y prueba, siendo el 80% para entrenamiento y el 20% para prueba.

In [199]:
# Se selecciona la variable objetivo, en este caso "sentimiento".
Y = df_peliculas['sentimiento']

# Del conjunto de datos se elimina la variable "processed_text"
X = df_peliculas.drop(['sentimiento'], axis=1)

# Se selecciona la variable "processed_text"
X = X['processed_text']

# Dividir los datos en entrenamiento y test
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

In [200]:
X_train.shape, Y_train.shape

((3498,), (3498,))

In [201]:
X_test.shape, Y_test.shape

((1500,), (1500,))

Vectorizar los comentarios de películas utilizando el método CountVectorizer

In [202]:
def tokenize(text):
    tokenizer = nltk.TweetTokenizer()
    return tokenizer.tokenize(text)

# Creamos el objeto CountVectorizer
count_vect = CountVectorizer(tokenizer=tokenize, stop_words=stop_words)

# Transformamos el conjunto de entrenamiento
X_train = count_vect.fit_transform(X_train)

# transformamos el conjunto de test
X_test = count_vect.transform(X_test)


Entrenar el modelo

In [203]:
nb = MultinomialNB()
nb.fit(X_train, Y_train)

MultinomialNB()

Predecir las etiquetas de los comentarios de prueba

In [204]:
Y_pred = nb.predict(X_test)

Calcular la precisión del modelo

In [205]:
print('Exactitud: %.2f' % accuracy_score(Y_test, Y_pred))
print("Recall: {}".format(recall_score(Y_test,Y_pred)))
print("Precisión: {}".format(precision_score(Y_test,Y_pred)))
print("Puntuación F1: {}".format(f1_score(Y_test,Y_pred)))

Exactitud: 0.82
Recall: 0.7575360419397117
Precisión: 0.8639760837070254
Puntuación F1: 0.8072625698324023


Reporte de clasificacion


In [206]:
print(classification_report(Y_test, Y_pred))

              precision    recall  f1-score   support

           0       0.78      0.88      0.82       737
           1       0.86      0.76      0.81       763

    accuracy                           0.82      1500
   macro avg       0.82      0.82      0.82      1500
weighted avg       0.82      0.82      0.82      1500



### Interpretación:

El modelo presenta unos buenos puntajes en las metricas de rendimiento y podria bueno para el cumplimiento de los objetivos del proyecto. Sin embargo, es importante tener en cuenta que el Modelo Naive Bayes asume la independencia condicional entre las características de los datos, lo que puede no ser cierto para el lenguaje natural. Por lo tanto, aunque el modelo puede funcionar bien para clasificar comentarios de películas como positivos o negativos, la precisión del modelo puede verse afectada por factores como la ambigüedad en el lenguaje y el sarcasmo.